In [1]:
from pathlib import Path
import requests
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# =========================
# PROJECT PATHS
# =========================
PROJECT_ROOT = Path.cwd()

DATA_DIR = PROJECT_ROOT / "data"
RAW_RECENT_DIR = DATA_DIR / "raw" / "recent"
RESULTS_DIR = DATA_DIR / "results"

TABLES_DIR = RESULTS_DIR / "tables"
FIGURES_DIR = RESULTS_DIR / "figures"

RAW_RECENT_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)


In [3]:
API_KEY = "e3273ecc2ae89542944197d0bff87f80af57f713087ddabb21bbb89a4a4c1a1f"
URL = "https://min-api.cryptocompare.com/data/v2/histohour"
HEADERS = {"authorization": f"Apikey {API_KEY}"}

COINS = {
    "BTC": "Bitcoin",
    "ETH": "Ethereum",
    "DASH": "Dash",
    "LTC": "Litecoin",
    "MAID": "MaidSafeCoin",
    "XMR": "Monero",
    "XRP": "Ripple"
}

# Last 3 years
END_DATE = pd.Timestamp.utcnow().floor("H")
START_DATE = END_DATE - pd.DateOffset(years=3)
def api_histohour(symbol, limit=2000, to_ts=None):
    params = {"fsym": symbol, "tsym": "USD", "limit": limit}
    if to_ts is not None:
        params["toTs"] = to_ts

    r = requests.get(URL, params=params, headers=HEADERS, timeout=30)
    r.raise_for_status()
    j = r.json()

    if j.get("Response") != "Success":
        raise RuntimeError(f"API error for {symbol}: {j.get('Message')}")

    df = pd.DataFrame(j["Data"]["Data"])
    df["time"] = pd.to_datetime(df["time"], unit="s", utc=True)
    return df.sort_values("time").reset_index(drop=True)


def download_last_3y(symbol, start_date, end_date):
    chunks = []
    to_ts = int(end_date.timestamp())

    while True:
        chunk = api_histohour(symbol, to_ts=to_ts)
        chunks.append(chunk)

        if chunk["time"].min() <= start_date:
            break

        to_ts = int(chunk["time"].min().timestamp()) - 3600

    df = (
        pd.concat(chunks)
        .drop_duplicates(subset="time")
        .sort_values("time")
        .reset_index(drop=True)
    )

    return df[(df["time"] >= start_date) & (df["time"] <= end_date)]

print(f"Downloading hourly data from {START_DATE} to {END_DATE}")

for sym, name in COINS.items():
    print(f"\n{name} ({sym})")

    df = download_last_3y(sym, START_DATE, END_DATE)

    out_path = RAW_RECENT_DIR / f"{sym}_USD_hourly_last3y.csv"
    df.to_csv(out_path, index=False)

    print(f"Saved → {out_path}")
    print(f"Rows: {len(df)}")


/var/folders/5p/qkrf3s2s08q9rspfsbwy6_kw0000gn/T/ipykernel_18412/1867241767.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  END_DATE = pd.Timestamp.utcnow().floor("H")



Bitcoin (BTC)
Saved → /Users/matildeventuri/Documents/GitHub/crypto-replicating-strategy/notebooks/data/raw/recent/BTC_USD_hourly_last3y.csv
Rows: 26305

Ethereum (ETH)
Saved → /Users/matildeventuri/Documents/GitHub/crypto-replicating-strategy/notebooks/data/raw/recent/ETH_USD_hourly_last3y.csv
Rows: 26305

Dash (DASH)
Saved → /Users/matildeventuri/Documents/GitHub/crypto-replicating-strategy/notebooks/data/raw/recent/DASH_USD_hourly_last3y.csv
Rows: 26305

Litecoin (LTC)
Saved → /Users/matildeventuri/Documents/GitHub/crypto-replicating-strategy/notebooks/data/raw/recent/LTC_USD_hourly_last3y.csv
Rows: 26305

MaidSafeCoin (MAID)
Saved → /Users/matildeventuri/Documents/GitHub/crypto-replicating-strategy/notebooks/data/raw/recent/MAID_USD_hourly_last3y.csv
Rows: 26305

Monero (XMR)
Saved → /Users/matildeventuri/Documents/GitHub/crypto-replicating-strategy/notebooks/data/raw/recent/XMR_USD_hourly_last3y.csv
Rows: 26305

Ripple (XRP)
Saved → /Users/matildeventuri/Documents/GitHub/crypto-r

In [4]:
stats = {
    "Minimum": [],
    "Q1": [],
    "Median": [],
    "Mean": [],
    "Q3": [],
    "Maximum": [],
    "SD": [],
    "Skewness": [],
    "Kurtosis": [],
    "CV": [],
    "Range": [],
    "IQR": []
}

for sym in COINS.keys():
    df = pd.read_csv(RAW_RECENT_DIR / f"{sym}_USD_hourly_last3y.csv")
    p = df["close"]

    stats["Minimum"].append(p.min())
    stats["Q1"].append(p.quantile(0.25))
    stats["Median"].append(p.median())
    stats["Mean"].append(p.mean())
    stats["Q3"].append(p.quantile(0.75))
    stats["Maximum"].append(p.max())
    stats["SD"].append(p.std())
    stats["Skewness"].append(p.skew())
    stats["Kurtosis"].append(p.kurtosis())
    stats["CV"].append(100 * p.std() / p.mean())
    stats["Range"].append(p.max() - p.min())
    stats["IQR"].append(p.quantile(0.75) - p.quantile(0.25))

table1 = pd.DataFrame(stats, index=COINS.values()).T


In [5]:
plt.figure(figsize=(10, 6))

for sym, name in COINS.items():
    df = pd.read_csv(RAW_RECENT_DIR / f"{sym}_USD_hourly_last3y.csv")
    prices = df["close"]
    indexed = prices / prices.iloc[0]
    plt.plot(indexed.values, label=name)

plt.xlabel("Hours")
plt.ylabel("Indexed Price")
plt.legend()
plt.tight_layout()

plt.savefig(
    FIGURES_DIR / "fig1_indexed_prices_last3y.png",
    dpi=300
)
plt.close()

print("Figure 1 saved in data/results/figures/")


Figure 1 saved in data/results/figures/


In [7]:
#strategia per Bitcoin 
from pathlib import Path
import pandas as pd
import numpy as np

PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
RAW_RECENT_DIR = DATA_DIR / "raw" / "recent"

# Load BTC data
df = pd.read_csv(RAW_RECENT_DIR / "BTC_USD_hourly_last3y.csv")

# Indexed price P_t (P0 = 1)
df["P"] = df["close"] / df["close"].iloc[0]

def compute_ema(series, n):
    alpha = 1 / n
    ema = [series.iloc[0]]
    for t in range(1, len(series)):
        ema.append(alpha * series.iloc[t] + (1 - alpha) * ema[t-1])
    return pd.Series(ema, index=series.index)

short_windows = [8, 16, 32]
long_windows  = [24, 48, 96]

for k, (ns, nl) in enumerate(zip(short_windows, long_windows), start=1):
    df[f"EMA_s_{k}"] = compute_ema(df["P"], ns)
    df[f"EMA_l_{k}"] = compute_ema(df["P"], nl)
    df[f"x_{k}"] = df[f"EMA_s_{k}"] - df[f"EMA_l_{k}"]

#normalizzazione 
t1 = 12
df["sigma_P"] = df["P"].rolling(window=t1).std()

for k in [1, 2, 3]:
    df[f"y_{k}"] = np.where(
        (df[f"x_{k}"] == 0) | (df["sigma_P"] == 0),
        0,
        df[f"x_{k}"] / df["sigma_P"]
    )

t2 = 168

#seconda normalizzazione
for k in [1, 2, 3]:
    sigma_y = df[f"y_{k}"].rolling(window=t2).std()
    df[f"z_{k}"] = np.where(
        (df[f"y_{k}"] == 0) | (sigma_y == 0),
        0,
        df[f"y_{k}"] / sigma_y
    )

#scaling non lineare
for k in [1, 2, 3]:
    df[f"u_{k}"] = df[f"z_{k}"] * np.exp(-(df[f"z_{k}"]**2) / 4)

#momentum signal
df["momentum_signal"] = (df["u_1"] + df["u_2"] + df["u_3"]) / 3


In [9]:
# =========================
# IMPORTS
# =========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# =========================
# RETURNS
# =========================
df["log_return"] = np.log(df["close"] / df["close"].shift(1))

# =========================
# POSITIONS (LONG / CASH)
# =========================
df["position"] = np.where(df["momentum_signal"] > 0, 1, 0)

# Avoid look-ahead bias
df["position_shifted"] = df["position"].shift(1).fillna(0)

# =========================
# STRATEGY & BENCHMARK RETURNS
# =========================
df["strategy_return"] = df["position_shifted"] * df["log_return"]
df["bh_return"] = df["log_return"]

# =========================
# CUMULATIVE RETURNS
# =========================
df["cum_strategy"] = df["strategy_return"].cumsum()
df["cum_bh"] = df["bh_return"].cumsum()

# =========================
# FIGURE: STRATEGY VS BUY-AND-HOLD
# =========================
FIGURES_DIR = Path("data/results/figures")
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

plt.figure(figsize=(10, 6))
plt.plot(df["cum_bh"], label="Buy-and-Hold", linestyle="--")
plt.plot(df["cum_strategy"], label="Momentum Strategy")

plt.xlabel("Hours")
plt.ylabel("Cumulative Log Returns")
plt.legend()
plt.tight_layout()

plt.savefig(
    FIGURES_DIR / "fig3_cumulative_returns_btc.png",
    dpi=300
)
plt.close()

# =========================
# PERFORMANCE METRICS
# =========================
def sharpe_ratio(returns, periods_per_year=24 * 365):
    mu = returns.mean()
    sigma = returns.std()
    return np.sqrt(periods_per_year) * mu / sigma if sigma != 0 else 0


def max_drawdown(cum_returns):
    running_max = cum_returns.cummax()
    drawdown = cum_returns - running_max
    return drawdown.min()

# =========================
# PERFORMANCE TABLE
# =========================
performance = pd.DataFrame({
    "Strategy": ["Momentum", "Buy-and-Hold"],
    "Mean Return": [
        df["strategy_return"].mean(),
        df["bh_return"].mean()
    ],
    "Volatility": [
        df["strategy_return"].std(),
        df["bh_return"].std()
    ],
    "Sharpe Ratio": [
        sharpe_ratio(df["strategy_return"]),
        sharpe_ratio(df["bh_return"])
    ],
    "Max Drawdown": [
        max_drawdown(df["cum_strategy"]),
        max_drawdown(df["cum_bh"])
    ],
    "% Time Invested": [
        df["position"].mean() * 100,
        100.0
    ]
})

# =========================
# SAVE TABLE
# =========================
TABLES_DIR = Path("data/results/tables")
TABLES_DIR.mkdir(parents=True, exist_ok=True)

performance.to_csv(
    TABLES_DIR / "table_performance_btc.csv",
    index=False,
    float_format="%.4f"
)

print("✔ Strategy applied successfully")
print("✔ Figure saved in data/results/figures/")
print("✔ Performance table saved in data/results/tables/")



✔ Strategy applied successfully
✔ Figure saved in data/results/figures/
✔ Performance table saved in data/results/tables/


In [10]:
# =========================
# IMPORTS
# =========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# =========================
# RETURNS
# =========================
df["log_return"] = np.log(df["close"] / df["close"].shift(1))

# =========================
# POSITIONS: LONG / SHORT
# =========================
# +1 = long
# -1 = short
df["position"] = np.where(
    df["momentum_signal"] > 0,  1,
    np.where(df["momentum_signal"] < 0, -1, 0)
)

# Avoid look-ahead bias
df["position_shifted"] = df["position"].shift(1).fillna(0)

# =========================
# STRATEGY & BUY-AND-HOLD RETURNS
# =========================
df["strategy_return"] = df["position_shifted"] * df["log_return"]
df["bh_return"] = df["log_return"]

# =========================
# CUMULATIVE RETURNS
# =========================
df["cum_strategy"] = df["strategy_return"].cumsum()
df["cum_bh"] = df["bh_return"].cumsum()

# =========================
# FIGURE: CUMULATIVE RETURNS
# =========================
FIGURES_DIR = Path("data/results/figures")
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

plt.figure(figsize=(10, 6))
plt.plot(df["cum_bh"], label="Buy-and-Hold", linestyle="--")
plt.plot(df["cum_strategy"], label="Momentum Long–Short")

plt.xlabel("Hours")
plt.ylabel("Cumulative Log Returns")
plt.legend()
plt.tight_layout()

plt.savefig(
    FIGURES_DIR / "fig3_cumulative_returns_btc_long_short.png",
    dpi=300
)
plt.close()

# =========================
# PERFORMANCE METRICS
# =========================
def sharpe_ratio(returns, periods_per_year=24*365):
    mu = returns.mean()
    sigma = returns.std()
    return np.sqrt(periods_per_year) * mu / sigma if sigma != 0 else 0


def max_drawdown(cum_returns):
    running_max = cum_returns.cummax()
    drawdown = cum_returns - running_max
    return drawdown.min()

# =========================
# PERFORMANCE TABLE
# =========================
performance = pd.DataFrame({
    "Strategy": ["Momentum Long–Short", "Buy-and-Hold"],
    "Mean Return": [
        df["strategy_return"].mean(),
        df["bh_return"].mean()
    ],
    "Volatility": [
        df["strategy_return"].std(),
        df["bh_return"].std()
    ],
    "Sharpe Ratio": [
        sharpe_ratio(df["strategy_return"]),
        sharpe_ratio(df["bh_return"])
    ],
    "Max Drawdown": [
        max_drawdown(df["cum_strategy"]),
        max_drawdown(df["cum_bh"])
    ],
    "% Time Long": [
        (df["position"] == 1).mean() * 100,
        100
    ],
    "% Time Short": [
        (df["position"] == -1).mean() * 100,
        0
    ]
})

# =========================
# SAVE TABLE
# =========================
TABLES_DIR = Path("data/results/tables")
TABLES_DIR.mkdir(parents=True, exist_ok=True)

performance.to_csv(
    TABLES_DIR / "table_performance_btc_long_short.csv",
    index=False,
    float_format="%.4f"
)

print("✔ Long–Short strategy applied")
print("✔ Figure saved (long–short)")
print("✔ Performance table saved")


✔ Long–Short strategy applied
✔ Figure saved (long–short)
✔ Performance table saved


In [20]:
# ======================================================
# FULL MOMENTUM STRATEGY – TIME SERIES & CROSS SECTIONAL
# ======================================================

import numpy as np
import pandas as pd
from pathlib import Path

# =========================
# CONFIG
# =========================
CRYPTO_LIST = ["BTC", "ETH", "DASH", "LTC", "MAID", "XMR", "XRP"]
DATA_DIR = Path("data/raw/recent")

short_windows = [8, 16, 32]
long_windows  = [24, 48, 96]
t1 = 12     # short volatility window
t2 = 168    # long volatility window

# =========================
# HELPERS
# =========================
def compute_ema(series, n):
    alpha = 1 / n
    ema = [series.iloc[0]]
    for t in range(1, len(series)):
        ema.append(alpha * series.iloc[t] + (1 - alpha) * ema[t-1])
    return pd.Series(ema, index=series.index)


# =========================
# LOAD DATA + MOMENTUM SIGNAL
# =========================
dfs = {}

for sym in CRYPTO_LIST:
    df = pd.read_csv(DATA_DIR / f"{sym}_USD_hourly_last3y.csv")

    # indexed price
    df["P"] = df["close"] / df["close"].iloc[0]

    # EMA-based base signals x_k
    for k, (ns, nl) in enumerate(zip(short_windows, long_windows), start=1):
        df[f"EMA_s_{k}"] = compute_ema(df["P"], ns)
        df[f"EMA_l_{k}"] = compute_ema(df["P"], nl)
        df[f"x_{k}"] = df[f"EMA_s_{k}"] - df[f"EMA_l_{k}"]

    # first normalization y_k
    df["sigma_P"] = df["P"].rolling(t1).std()

    for k in [1, 2, 3]:
        df[f"y_{k}"] = np.where(
            (df[f"x_{k}"] == 0) | (df["sigma_P"] == 0),
            0,
            df[f"x_{k}"] / df["sigma_P"]
        )

    # second normalization z_k
    for k in [1, 2, 3]:
        sigma_y = df[f"y_{k}"].rolling(t2).std()
        df[f"z_{k}"] = np.where(
            (df[f"y_{k}"] == 0) | (sigma_y == 0),
            0,
            df[f"y_{k}"] / sigma_y
        )

    # scaling u_k
    for k in [1, 2, 3]:
        df[f"u_{k}"] = df[f"z_{k}"] * np.exp(-(df[f"z_{k}"]**2) / 4)

    # final momentum signal
    df["momentum_signal"] = (df["u_1"] + df["u_2"] + df["u_3"]) / 3

    # returns
    df["log_return"] = np.log(df["close"] / df["close"].shift(1))

    dfs[sym] = df


# =========================
# ALIGN SIGNALS & RETURNS
# =========================
signals = pd.DataFrame({sym: dfs[sym]["momentum_signal"] for sym in CRYPTO_LIST})
returns = pd.DataFrame({sym: dfs[sym]["log_return"] for sym in CRYPTO_LIST})

signals = signals.shift(1)   # no look-ahead

# =========================
# TIME-SERIES PORTFOLIO
# =========================
N = len(CRYPTO_LIST)
ts_returns = (signals / N * returns).sum(axis=1)
ts_cum = ts_returns.cumsum()

# =========================
# CROSS-SECTIONAL PORTFOLIO
# =========================
def cross_sectional_returns(signals, returns):
    out = []

    for t in range(1, len(signals)):   # start from t = 1
        s = signals.iloc[t-1]          # signal at t-1
        r = returns.iloc[t]            # return at t

        if s.isna().any() or r.isna().any():
            out.append(0)
            continue

        ranked = s.sort_values()

        short_assets = ranked.index[:3]
        long_assets  = ranked.index[-3:]

        ret = (r[long_assets] / 6).sum() - (r[short_assets] / 6).sum()
        out.append(ret)

    return pd.Series([0] + out, index=signals.index)

cs_returns = cross_sectional_returns(signals, returns)
cs_cum = cs_returns.cumsum()


print("✔ Momentum signals computed for all cryptocurrencies")
print("✔ Time-series and cross-sectional portfolios constructed")


✔ Momentum signals computed for all cryptocurrencies
✔ Time-series and cross-sectional portfolios constructed


In [21]:
# =========================
# PLOTS: PORTFOLIO PERFORMANCE
# =========================
import matplotlib.pyplot as plt
from pathlib import Path

FIGURES_DIR = Path("data/results/figures")
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# =========================
# FIGURE 4: TIME-SERIES MOMENTUM
# =========================
plt.figure(figsize=(10, 6))

plt.plot(ts_cum, label="Time-Series Momentum Portfolio")

plt.xlabel("Hours")
plt.ylabel("Cumulative Log Returns")
plt.legend()
plt.tight_layout()

plt.savefig(
    FIGURES_DIR / "fig4_time_series_momentum.png",
    dpi=300
)
plt.close()

# =========================
# FIGURE 5: CROSS-SECTIONAL MOMENTUM
# =========================
plt.figure(figsize=(10, 6))

plt.plot(cs_cum, label="Cross-Sectional Momentum Portfolio")

plt.xlabel("Hours")
plt.ylabel("Cumulative Log Returns")
plt.legend()
plt.tight_layout()

plt.savefig(
    FIGURES_DIR / "fig5_cross_sectional_momentum.png",
    dpi=300
)
plt.close()

print("✔ Plots saved in data/results/figures/")


✔ Plots saved in data/results/figures/


In [19]:
signals.describe()


,BTC,ETH,DASH,LTC,MAID,XMR,XRP
count,26127.000000,26127.000000,26127.000000,26127.000000,26127.000000,26127.000000,26127.000000
mean,0.054865,0.015743,-0.009428,0.008608,-0.015114,0.069143,-0.021099
std,0.498449,0.495674,0.514873,0.501269,0.426814,0.514055,0.508790
min,-0.857479,-0.856502,-0.857453,-0.856563,-0.857591,-0.857500,-0.857511
25%,-0.382086,-0.423800,-0.477432,-0.432305,-0.326669,-0.384807,-0.483164
50%,0.078119,0.030226,-0.009610,-0.009039,-0.016149,0.132251,-0.037396
75%,0.508033,0.448805,0.463521,0.466363,0.296111,0.523073,0.432869
max,0.856621,0.857604,0.856996,0.857247,0.856984,0.857619,0.857030


In [22]:
# ======================================================
# CROSS-SECTIONAL CONTRARIAN STRATEGY (FULL PIPELINE)
# ======================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# =========================
# CONFIG
# =========================
CRYPTO_LIST = ["BTC", "ETH", "DASH", "LTC", "MAID", "XMR", "XRP"]
DATA_DIR = Path("data/raw/recent")

short_windows = [8, 16, 32]
long_windows  = [24, 48, 96]
t1 = 12      # short volatility window
t2 = 168     # long volatility window

# =========================
# HELPERS
# =========================
def compute_ema(series, n):
    alpha = 1 / n
    ema = [series.iloc[0]]
    for t in range(1, len(series)):
        ema.append(alpha * series.iloc[t] + (1 - alpha) * ema[t-1])
    return pd.Series(ema, index=series.index)

# =========================
# LOAD DATA + SIGNALS
# =========================
dfs = {}

for sym in CRYPTO_LIST:
    df = pd.read_csv(DATA_DIR / f"{sym}_USD_hourly_last3y.csv")

    # indexed price
    df["P"] = df["close"] / df["close"].iloc[0]

    # EMA base signals
    for k, (ns, nl) in enumerate(zip(short_windows, long_windows), start=1):
        df[f"EMA_s_{k}"] = compute_ema(df["P"], ns)
        df[f"EMA_l_{k}"] = compute_ema(df["P"], nl)
        df[f"x_{k}"] = df[f"EMA_s_{k}"] - df[f"EMA_l_{k}"]

    # first normalization
    df["sigma_P"] = df["P"].rolling(t1).std()
    for k in [1, 2, 3]:
        df[f"y_{k}"] = np.where(
            (df[f"x_{k}"] == 0) | (df["sigma_P"] == 0),
            0,
            df[f"x_{k}"] / df["sigma_P"]
        )

    # second normalization
    for k in [1, 2, 3]:
        sigma_y = df[f"y_{k}"].rolling(t2).std()
        df[f"z_{k}"] = np.where(
            (df[f"y_{k}"] == 0) | (sigma_y == 0),
            0,
            df[f"y_{k}"] / sigma_y
        )

    # scaling
    for k in [1, 2, 3]:
        df[f"u_{k}"] = df[f"z_{k}"] * np.exp(-(df[f"z_{k}"] ** 2) / 4)

    # final momentum signal
    df["momentum_signal"] = (df["u_1"] + df["u_2"] + df["u_3"]) / 3

    # returns
    df["log_return"] = np.log(df["close"] / df["close"].shift(1))

    dfs[sym] = df

# =========================
# BUILD SIGNALS & RETURNS
# =========================
signals = pd.DataFrame({sym: dfs[sym]["momentum_signal"] for sym in CRYPTO_LIST})
returns = pd.DataFrame({sym: dfs[sym]["log_return"] for sym in CRYPTO_LIST})

# lag signals (no look-ahead)
signals_lag = signals.shift(1)

# =========================
# CROSS-SECTIONAL CONTRARIAN
# =========================
def cs_contrarian_returns(signals_lag, returns):
    out = []
    for t in range(1, len(signals_lag)):
        s = signals_lag.iloc[t]
        r = returns.iloc[t]

        if s.isna().any() or r.isna().any():
            out.append(0.0)
            continue

        ranked = s.sort_values()

        long_assets  = ranked.index[:3]    # bottom 3
        short_assets = ranked.index[-3:]   # top 3

        ret = (r[long_assets] / 6).sum() - (r[short_assets] / 6).sum()
        out.append(ret)

    return pd.Series([0.0] + out, index=signals_lag.index)

cs_contra_ret = cs_contrarian_returns(signals_lag, returns)
cs_contra_cum = cs_contra_ret.cumsum()

# =========================
# PLOT
# =========================
FIGURES_DIR = Path("data/results/figures")
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

plt.figure(figsize=(10, 6))
plt.plot(cs_contra_cum, label="Cross-Sectional Contrarian Strategy")
plt.xlabel("Hours")
plt.ylabel("Cumulative Log Returns")
plt.legend()
plt.tight_layout()
plt.savefig(FIGURES_DIR / "fig_cs_contrarian.png", dpi=300)
plt.close()

# =========================
# PERFORMANCE TABLE
# =========================
def sharpe_ratio(returns, periods_per_year=24*365):
    mu = returns.mean()
    sigma = returns.std()
    return np.sqrt(periods_per_year) * mu / sigma if sigma != 0 else 0

def max_drawdown(cum_returns):
    running_max = cum_returns.cummax()
    drawdown = cum_returns - running_max
    return drawdown.min()

performance = pd.DataFrame({
    "Strategy": ["CS Contrarian"],
    "Mean Return": [cs_contra_ret.mean()],
    "Volatility": [cs_contra_ret.std()],
    "Sharpe Ratio": [sharpe_ratio(cs_contra_ret)],
    "Max Drawdown": [max_drawdown(cs_contra_cum)]
})

TABLES_DIR = Path("data/results/tables")
TABLES_DIR.mkdir(parents=True, exist_ok=True)

performance.to_csv(
    TABLES_DIR / "table_performance_cs_contrarian.csv",
    index=False,
    float_format="%.6f"
)

print("✔ Strategy executed successfully")
print("✔ Plot saved in data/results/figures/")
print("✔ Table saved in data/results/tables/")
print("Final cumulative log return:", float(cs_contra_cum.iloc[-1]))


✔ Strategy executed successfully
✔ Plot saved in data/results/figures/
✔ Table saved in data/results/tables/
Final cumulative log return: 6.148323117509149
